# Install library

In [1]:
!pip install rdflib
!pip install PyDrive

     |████████████████████████████████| 235kB 6.6MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 


# Import library


In [2]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS
from rdflib.serializer import Serializer

In [32]:
graph = Graph()
graph.parse("onto_xml.owl", format="application/rdf+xml")
graph.serialize(format='n3')

b'@prefix : <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n:Bad a owl:Class ;\n    rdfs:subClassOf :Status .\n\n:Boevik a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Detektivy a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Fanfik a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Fantastika a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Fentezi a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Good a owl:Class ;\n    rdfs:subClassOf :Status .\n\n:Istoricheskiy_roman a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Jenskaya_proza a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Litrpg a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Lyubovnye_romany a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Mistikaujasy a owl:Class ;\n    rdfs:subClassOf :Genre .\n\n:Molodejnaya_proza a owl:Class ;\n    rdfs:subClassOf :Ge

# Class

In [33]:
path = 'http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#'

# Class
TAGS = URIRef(path + 'Tags')
BOOK = URIRef(path + 'Book')

STATUS_BAD = URIRef(path + 'Bad')
STATUS_GOOD = URIRef(path + 'Good')
STATUS_SATISFACTORY = URIRef(path + 'Satisfactory')
GENRE = URIRef(path + 'Genre')
AUTHOR = URIRef(path + 'Author')
EDUCATIONAL_BOOK = URIRef(path + 'Educational_book')
ENTERTAIMENT_BOOK = URIRef(path + 'Entertainment_book')

NAME_GENRES = {
    'Фэнтези': URIRef(path + 'Fentezi'),
    'ЛитРПГ': URIRef(path + 'Litrpg'),
    'Любовные романы': URIRef(path + 'Lyubovnye_romany'),
    'Фантастика': URIRef(path + 'Fantastika'),
    'Исторический роман': URIRef(path + 'Istoricheskiy_roman'),
    'Молодежная проза': URIRef(path + 'Molodejnaya_proza'),
    'Попаданцы': URIRef(path + 'Popadancy'),
    'Женский роман': URIRef(path + 'Jenskaya_proza'),
    'Фанфик': URIRef(path + 'Fanfik'),
    'Детективы': URIRef(path + 'Detektivy'),
    'Проза': URIRef(path + 'Proza'),
    'Боевик': URIRef(path + 'Boevik'),
    'Триллеры': URIRef(path + 'Trillery'),
    'Мистика/Ужасы': URIRef(path + 'Mistikaujasy'),
    'Разное': URIRef(path + 'Raznoe'),
}

# Object Property

In [34]:
# Object Property
HAS_STATUS = URIRef(path + 'hasStatus')
HAS_AUTHOR = URIRef(path + 'hasAuthor')
HAS_GENRE = URIRef(path + 'hasGenre')
HAS_TAGS = URIRef(path + 'hasTags')

# Data Property

In [35]:
# Data Property
HAS_RATE = URIRef(path + 'hasRate')
HAS_FAVORITES = URIRef(path + 'hasFavorites')
HAS_VIEWS = URIRef(path + 'hasViews')
HAS_NAME = URIRef(path + 'hasName')
HAS_URL = URIRef(path + 'hasURL')
HAS_NUMBER_OF_PAGES = URIRef(path + 'hasNumberOfPages')
HAS_DESCRIPTION = URIRef(path + 'hasDescription')
HAS_DATE_OF_PUBLICATION = URIRef(path + 'hasDateOfPublication')

# Loads data

## Get data from JSON file


In [36]:
import json

def get_data_json(path: str):
  with open(path, 'r') as f:
    data = json.load(f)
    return data

data = get_data_json('data.json')
data[0]

{'author': [{'author_name': 'Лира Алая',
   'author_page': 'https://litnet.com/ru/lira-alaya-u530349'}],
 'book_tags': ['бытовое фэнтези',
  'неунывающая героиня',
  'говорящая таблеточная машина'],
 'book_url': 'https://litnet.com/ru/book/tabletochku-vashe-temneishestvo-b333552',
 'description': '\nТаблеточку, Ваше Темнейшество?\nЛира Алая\n                Прислонилась к таблеточной машине и попала в другой мир? Не теряйся! Провизор ты или кто? Зря латынь учила, что ли? Сойдёшь за ведьму! Аптеку откроешь.\r\nСветлые тут, ко... подробнее\nВ тексте есть:  бытовое фэнтези, неунывающая героиня, говорящая таблеточная машина\n\n\n\ue86a\n                        В процессе: 15 Мая\n                        55 стр\n\n\n278441\n19292\n\n\n',
 'favourites': 19292,
 'genre': 'Фэнтези',
 'number_pages': 55,
 'rating': 2330,
 'title': 'Таблеточку, Ваше Темнейшество?',
 'type_book': 'Entertainment_book',
 'views': 278441}

## Add data in graph

In [37]:
from rdflib import Literal, XSD, DCTERMS
import re

def replace_space(text):
  text = re.findall(r'[\w]+', text)
  return '_'.join(text).rstrip()


def add_author(authors: list, book_uri):
  for author in authors:
    author_name = URIRef(path + replace_space(author['author_name']))

    graph.add((author_name, RDF.type, AUTHOR))
    graph.add((author_name, HAS_URL, Literal(author['author_page'])))
    graph.add((author_name, HAS_AUTHOR, book_uri))


def add_genre(genre: str, book_uri):
  graph.add((book_uri, RDF.type, NAME_GENRES[genre]))
  graph.add((book_uri, HAS_GENRE, NAME_GENRES[genre]))
  graph.add((NAME_GENRES[genre], HAS_NAME, Literal(genre, datatype=XSD.string)))


def add_views(views: int, book_uri):
  graph.add((book_uri, HAS_VIEWS, Literal(views, datatype=XSD.integer)))


def add_number_pages(cnt_page: int, book_uri):
  graph.add((book_uri, HAS_NUMBER_OF_PAGES, Literal(cnt_page, datatype=XSD.integer)))


def add_favorites(favorites: int, book_uri):
  graph.add((book_uri, HAS_FAVORITES, Literal(favorites, datatype=XSD.integer)))


def add_description(desc: str, book_uri):
  graph.add((book_uri, HAS_DESCRIPTION, Literal(desc, datatype=XSD.string)))


def add_status(rate: int, book_uri):
  if rate > 5000:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_GOOD))
    graph.add((STATUS_GOOD, HAS_STATUS, book_uri))

  elif rate > 1000 and rate <= 5000:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_SATISFACTORY))
    graph.add((STATUS_SATISFACTORY, HAS_STATUS, book_uri))
  else:
    graph.add((book_uri, HAS_RATE, Literal(rate, datatype=XSD.integer)))
    graph.add((book_uri, RDF.type, STATUS_BAD))
    graph.add((STATUS_BAD, HAS_STATUS, book_uri))



def add_book_tags(tags: list, book_uri):
  for tag in tags:
    tag = URIRef(path + replace_space(tag))
    graph.add((tag, RDF.type, TAGS))
    graph.add((tag, HAS_NAME, Literal(tag, datatype=XSD.string)))
    graph.add((tag, HAS_TAGS, book_uri))


def add_book(book: dict):
  book_uri = URIRef(path + replace_space(book['title']))

  if book['type_book'] == 'Entertainment_book':
    graph.add((book_uri, RDF.type, ENTERTAIMENT_BOOK))
  else:
    graph.add((book_uri, RDF.type, EDUCATIONAL_BOOK))

  graph.add((book_uri, HAS_NAME, Literal(book['title'], datatype=XSD.string)))
  graph.add((book_uri, HAS_URL, Literal(book['book_url'], datatype=XSD.string)))

  add_status(book['rating'], book_uri)
  add_number_pages(book['number_pages'], book_uri)
  add_author(book['author'], book_uri)
  add_book_tags(book['book_tags'], book_uri)
  add_genre(book['genre'], book_uri)
  add_views(book['views'], book_uri)
  add_favorites(book['favourites'], book_uri)
  add_description(book['description'], book_uri)


In [38]:
for el in data:
  add_book(el)

## Save data in .owl file

In [46]:
graph.serialize("book_ontology.rdf", format="application/rdf+xml")

# Query in ontology

In [40]:
# Топ 10 популярных книг 
qv1= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?r
    WHERE {
      ?b rdf:type path:Entertainment_book .
      ?b path:hasRate ?r .
      ?b path:hasName ?y
      FILTER (?r > 20000)
    } LIMIT 10
""")
for row in qv1:
    print("%s %s" % row)

Смертельный способ выйти замуж 24211
Искажающие реальность-2 20534
Попаданка в Академии драконов 20825
Я - Стрела. Отбор в академию Стражей 20054
Селфи на балконе 23621
Невеста на неделю, или Моя навеки 22353
Маленькое чудо для злодея 28704
Мой (не)любимый дракон. Книга 2 20130
Невеста из другого мира 2. Свет Полуночи 27945
Зачем я ему? 27010


In [41]:
# У каких книг жанра X количество просмотров превышает Y?
qv2= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?v
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b path:hasGenre path:Boevik .
    ?b path:hasViews ?v . 
    ?b path:hasName ?y
    FILTER (?v > 500000)
  }""")
for row in qv2:
    print("%s %s" % row)

Задача выжить 725034
Новое Начало. Книга Первая 792966
Серый кукловод. Часть 1 712620
Новое Начало. Книга Вторая 502248


In [42]:
# Наиболее часто добавляемые книги в библиотеку жанра боевик?
qv3= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?f
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b path:hasGenre path:Boevik .
    ?b path:hasFavorites ?f . 
    ?b path:hasName ?y
  }
  ORDER BY DESC(?f)
  LIMIT 10
  """)
for row in qv3:
    print("%s %s" % row)

Поварёшка 13040
Новое Начало. Книга Первая 11300
Серый кукловод. Часть 1 9534
Новое Начало. Книга Вторая 8835
Новое Начало. Книга Третья 7507
Колонизация 6469
Механист 5622
Задача выжить 4462
Арсанты. Дети богов. 4067
Падший 3629


In [43]:
# Какие книги статуса X чаще все добавляют в свою коллекцию?
qv4= graph.query("""
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX owl: <http://www.w3.org/2002/07/owl#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  PREFIX path: <http://www.semanticweb.org/vasilii/ontologies/2021/3/book_onto#>
  SELECT ?y ?f
  WHERE {
    ?b rdf:type path:Entertainment_book .
    ?b rdf:type path:Good .
    ?b path:hasFavorites ?f . 
    ?b path:hasName ?y
  } 
  ORDER BY DESC(?f)
  LIMIT 10
  """)

for row in qv4:
    print("%s %s" % row)

Опасный босс 80126
Мой лунный эльф, или Как не влюбиться по уши 59539
Взрывная смесь  58884
Меняю на нового ... или обмен по-русски 58524
Драконий подарок. Королевская академия Драко 57307
Попаданка в Академии драконов 49651
Невеста из другого мира 49081
Пленница лунного эльфа 45390
Смертельный способ выйти замуж 42686
Маленькое чудо для злодея 42621


# Generate documentation

In [18]:
!pip install pylode

     |████████████████████████████████| 143kB 6.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
  Created wheel for rdflib-jsonld: filename=rdflib_jsonld-0.5.0-py2.py3-none-any.whl size=15348 sha256=8cd8bb18537239d789fc3976303ba5f37829a719eeaf12f0d3ce40884f98e3a6
  Stored in directory: /root/.cache/pip/wheels/8d/e4/7f/9ebcb3e400c694e645c3adba40ef3e9bda78384ac3b9b0d13d
Successfully built rdflib-jsonld


In [19]:
import pylode

html = pylode.MakeDocco(input_data_file='book_onto3.owl', outputformat="html").document()

f = open('doc.html', 'w')
f.write(html)
f.close()

# VOID

In [44]:
from rdflib.void import generateVoID

prefix = dict(graph.namespaces())['']

graph.add((prefix, DCTERMS.title, Literal('Selection of books')))
graph.add((prefix, DCTERMS.publisher, Literal('Kuryshev Vasilii')))

void_graph, _ = generateVoID(graph)
print(void_graph.serialize(format='turtle').decode('utf-8'))

@prefix ns1: <http://rdfs.org/ns/void#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/Dataset> a ns1:Dataset ;
    ns1:classPartition <http://example.org/Dataset_class0>,
        <http://example.org/Dataset_class1>,
        <http://example.org/Dataset_class10>,
        <http://example.org/Dataset_class11>,
        <http://example.org/Dataset_class12>,
        <http://example.org/Dataset_class13>,
        <http://example.org/Dataset_class14>,
        <http://example.org/Dataset_class15>,
        <http://example.org/Dataset_class16>,
        <http://example.org/Dataset_class17>,
        <http://example.org/Dataset_class18>,
        <http://example.org/Dataset_class19>,
        <http://example.org/Dataset_class2>,
        <http://example.org/Dataset_class20>,
        <http://example.org/Dataset_class21>,
        <http://example.org/Dataset_class22>,
        <http://example.org/Dataset_class23>,
        <ht

# Reasoner

In [48]:
!pip install owlready2

     |████████████████████████████████| 23.7MB 146kB/s 
  Created wheel for owlready2: filename=Owlready2-0.31-cp37-cp37m-linux_x86_64.whl size=20409719 sha256=0f225bafd009c1c25dfc19db6a701ef75c95cd243f6107d24c754572df8f11a0
  Stored in directory: /root/.cache/pip/wheels/9d/17/e4/54fb8381c51746a0d686b06ef3e3f607789defd8e1580bcb1b
Successfully built owlready2


In [49]:
from owlready2 import *

onto = get_ontology('book_ontology.rdf')
with onto:
  sync_reasoner()


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/hermit:/usr/local/lib/python3.7/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpsoruzl22
* Owlready2 * HermiT took 1.5090694427490234 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
